In [1]:
import logging
from xbrl.cache import HttpCache
from xbrl.instance import XbrlParser, XbrlInstance
import json
from xbrl.linkbase import PresentationArc
from xbrl.instance import parse_xbrl_url
import xbrl
import pandas as pd
from os import listdir, path
from os.path import isfile, join
from importlib import reload

In [2]:
from digiaccounts_data import create_profitloss_table, get_startend_period, get_company_info

In [3]:
account_root = 'accounts_data'

all_accounts = [f for f in listdir(account_root) if isfile(join(account_root, f))]
all_accounts
active = [a for a in all_accounts if '07676340' in a]
active

['07676340_aa_2018_12_31.xhtml',
 '07676340_aa_2019_12_04.xhtml',
 '07676340_aa_2021-03-24.xhtml',
 '07676340_aa_2021-12-29.xhtml']

In [4]:
logging.basicConfig(level=logging.WARNING)

cache: HttpCache = HttpCache('./cache')
cache.set_headers({'From': 'ahoward@companieshouse.gov.uk', 'User-Agent': 'py-xbrl/2.1.0'})
parser = XbrlParser(cache)

schema = path.join(account_root, active[3])

inst: XbrlInstance = parser.parse_instance(schema)

In [5]:
for fact in inst.facts:
    if hasattr(fact.context, 'instant_date'):
        if hasattr(fact, 'unit'):
            print(fact.concept.name, f'£{fact.value}', f'date={fact.context.instant_date}\n')
            #print(fact.unit)
        else:
            print(fact.concept.name, fact.value, f'date={fact.context.instant_date}\n')
    else:
        print(fact.concept.name, fact.value, '\n')
    # if fact.concept.name == 'Creditors':
    #     print(dir(fact.context.segments))
    #     break

FixedAssets £12964.0 date=2021-03-31

FixedAssets £17285.0 date=2020-03-31

CurrentAssets £39112.0 date=2021-03-31

CurrentAssets £21842.0 date=2020-03-31

Creditors £19770.0 date=2021-03-31

Creditors £19036.0 date=2020-03-31

NetCurrentAssetsLiabilities £19342.0 date=2021-03-31

NetCurrentAssetsLiabilities £2806.0 date=2020-03-31

TotalAssetsLessCurrentLiabilities £32306.0 date=2021-03-31

TotalAssetsLessCurrentLiabilities £20091.0 date=2020-03-31

Creditors £25000.0 date=2021-03-31

Creditors £0.0 date=2020-03-31

AccruedLiabilitiesNotExpressedWithinCreditorsSubtotal £0.0 date=2021-03-31

AccruedLiabilitiesNotExpressedWithinCreditorsSubtotal £1165.0 date=2020-03-31

NetAssetsLiabilities £7306.0 date=2021-03-31

NetAssetsLiabilities £18926.0 date=2020-03-31

Equity £7306.0 date=2021-03-31

Equity £18926.0 date=2020-03-31

AverageNumberEmployeesDuringPeriod 4.0 

AverageNumberEmployeesDuringPeriod 4.0 

LegalFormEntity  

StartDateForPeriodCoveredByReport 2020-04-01 date=2020-03-31

E

In [81]:
for fact in inst.facts:
    if fact.concept.name == 'Creditors':
        print(fact.context.segments)
        break

In [74]:
for fact in inst.facts:
    if fact.concept.name == 'Creditors':
        x = fact.json()
        print(x)
        break


{'dimensions': {'unit': 'iso4217:GBP', 'concept': 'Creditors', 'entity': '07676340', 'period': '2021-03-31', 'MaturitiesOrExpirationPeriodsDimension': 'WithinOneYear'}, 'decimals': 0, 'value': 19770.0}


In [5]:
pl = create_profitloss_table(inst)
start, end = get_startend_period(inst)
reg_number, address = get_company_info(inst)

In [6]:
pl

,Fact,2020-03-31,2021-03-31
0,FixedAssets,17285.0,12964.0
1,CurrentAssets,21842.0,39112.0
2,CreditorsWithinOneYear,19036.0,19770.0
3,NetCurrentAssetsLiabilities,2806.0,19342.0
4,TotalAssetsLessCurrentLiabilities,20091.0,32306.0
5,CreditorsAfterOneYear,0.0,25000.0
6,AccruedLiabilitiesNotExpressedWithinCreditorsS...,1165.0,0.0
7,NetAssetsLiabilities,18926.0,7306.0
8,Equity,18926.0,7306.0


In [7]:
print(f'Period Start Date: {start}', 
      f'\nPeriod End Date: {end}')


Period Start Date: 2020-03-31 
Period End Date: 2021-03-31


In [8]:
print(f'Company Registration Number: {reg_number}')
print(f'Company Address: \n{address}')

Company Registration Number: 07676340
Company Address: 
{'AddressLine1': ['7 CAER STREET'], 'PrincipalLocation-CityOrTown': ['SWANSEA'], 'PostalCodeZip': ['SA1 3PP']}


In [9]:
pd.DataFrame(address)

,AddressLine1,PrincipalLocation-CityOrTown,PostalCodeZip
0,7 CAER STREET,SWANSEA,SA1 3PP


In [9]:
x = json.loads(inst.json())
x['facts']

{'f0': {'dimensions': {'unit': 'iso4217:GBP',
   'concept': 'FixedAssets',
   'entity': '07676340',
   'period': '2021-03-31'},
  'decimals': 0,
  'value': 12964.0},
 'f1': {'dimensions': {'unit': 'iso4217:GBP',
   'concept': 'FixedAssets',
   'entity': '07676340',
   'period': '2020-03-31'},
  'decimals': 0,
  'value': 17285.0},
 'f2': {'dimensions': {'unit': 'iso4217:GBP',
   'concept': 'CurrentAssets',
   'entity': '07676340',
   'period': '2021-03-31'},
  'decimals': 0,
  'value': 39112.0},
 'f3': {'dimensions': {'unit': 'iso4217:GBP',
   'concept': 'CurrentAssets',
   'entity': '07676340',
   'period': '2020-03-31'},
  'decimals': 0,
  'value': 21842.0},
 'f4': {'dimensions': {'unit': 'iso4217:GBP',
   'concept': 'Creditors',
   'entity': '07676340',
   'period': '2021-03-31',
   'MaturitiesOrExpirationPeriodsDimension': 'WithinOneYear'},
  'decimals': 0,
  'value': 19770.0},
 'f5': {'dimensions': {'unit': 'iso4217:GBP',
   'concept': 'Creditors',
   'entity': '07676340',
   'peri

In [6]:
s = 'foo'
array = ['foo', 'bar', 'dummy']
print(any(a in s for a in array))

True
